In [24]:
# Imports

#Dwave imports
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dimod
from __future__ import print_function

import dimod
import math
import sys
import copy

from dimod.generators.constraints import combinations
from hybrid.reference import KerberosSampler

#Impots for QUBO problem
import numpy as np
from numpy import linalg as LA
import pandas as pd
import matplotlib as plt
import itertools as it
from itertools import product
from matplotlib import pyplot as plt
import math

import matplotlib.pyplot as plt
import networkx as nx

import json
import pennylane as qml
import pennylane.numpy as np

In [25]:
# Code to find the set of indicies that are next to vertex p. Verticies are labeled numerically from 0 to N^2 - 1
# We note that the index can be encoded as a tuple via p = px*n+py, hence py = p mod n and px = (i - iy)/n

def next_to(p,n,T):
    if T == 'Default':
        py = int(p % n) # the y coordinate (y in {0,...,n-1})
        px = int((p - py)/n) # the x coordinate (x in {0,...,n-1})
        # print((px,py))
        adjacent_set = []
        if px + 1 < n and px + 1 >= 0:
            adjacent_set.append((px + 1, py))
        if px - 1 < n and px - 1 >= 0:
            adjacent_set.append((px - 1, py))
        if py + 1 < n and py + 1 >= 0:
            adjacent_set.append((px, py + 1))
        if py - 1 < n and py - 1 >= 0:
            adjacent_set.append((px, py - 1)) 
        tensors = []
        neighbours = []
        for x,y in adjacent_set:
            tensors.append(f'G_{x*n + y}')
            neighbours.append(x*n + y)
        return neighbours
    
    elif T == 'Periodic':
        py = int(p % n) # the y coordinate (y in {0,...,n-1})
        px = int((p - py)/n) # the x coordinate (x in {0,...,n-1})
        # print((px,py))
        adjacent_set = []
        adjacent_set.append(((px + 1) % n, py))
        adjacent_set.append(((px - 1) % n, py)) 
        adjacent_set.append((px, (py + 1) % n))
        adjacent_set.append((px, (py - 1) % n)) 
        tensors = []
        neighbours = []
        # print(f'G_{p}')
        for x,y in adjacent_set:
            # print((px,py),f'G_{x*n + y}')
            tensors.append(f'G_{x*n + y}')
            neighbours.append(x*n + y)

        return neighbours


    elif T == 'X Shift':
        shift = 1
        py = int(p % n) # the y coordinate (y in {0,...,n-1})
        px = int((p - py)/n) # the x coordinate (x in {0,...,n-1})
        # print((px,py))
        adjacent_set = []
        

        #Bulk
        if px not in [0,n-1]:
            adjacent_set.append(((px + 1) % n, py))
            adjacent_set.append(((px - 1) % n, py)) 
            adjacent_set.append((px, (py + 1) % n))
            adjacent_set.append((px, (py - 1) % n)) 
            # print('Bulk point: ',adjacent_set)
        #x lower boundary
        elif px == 0:
            adjacent_set.append(((px + 1) % n, py))
            adjacent_set.append(((px - 1) % n, (py - shift) % n)) 
            adjacent_set.append((px, (py + 1) % n))
            adjacent_set.append((px, (py - 1) % n))
            # print('lower x point: ', adjacent_set)
        #x upper boundary
        elif px == n-1:
            adjacent_set.append(((px + 1) % n, (py + shift) % n))
            adjacent_set.append(((px - 1) % n, py % n)) 
            adjacent_set.append((px, (py + 1) % n))
            adjacent_set.append((px, (py - 1) % n))
            # print('upper x point: ', adjacent_set)
        tensors = []
        neighbours = []
        # print(f'G_{p}')
        for x,y in adjacent_set:
            # print((px,py),f'G_{x*n + y}')
            tensors.append(f'G_{x*n + y}')
            neighbours.append(x*n + y)

        return neighbours

# Convert to coordiante form
def coords(p,n):
    p = int(p)
    py = int(p % n) # the y coordinate (y in {0,...,n-1})
    px = int((p - py)/n) # the x coordinate (x in {0,...,n-1})
    return (px,py)

# Get adjacent Edges for each point
def edge_set(p,n):
    neighbours = next_to(p,n, T = BC)
    edge_set = []
    edge_tensors = []
    for v in neighbours:
        if v < p:
            edge_set.append((v,p))
            edge_tensors.append(f'G_{v}_{p}')
        else:
            edge_set.append((p,v))
            edge_tensors.append(f'G_{p}_{v}')
    return edge_set

# Get all bonds globally
def get_all_bonds_global(n):
    bonds = []
    for j in range(n**2):
        for p in next_to(j,n, T = BC):
            bonds.append(bond_map(f'G_{j}_{p}'))
        bonds = list(set(bonds))
    return bonds

# Get all local bonds
def get_all_bonds(j,n):
    bonds = []
    for p in next_to(j,n, T = BC):
        bonds.append(bond_map(f'G_{j}_{p}'))
    bonds = list(set(bonds))
    return bonds

# Bond map G_j_i -> G_i_j where i < j
def bond_map(bond):
    # print(bond)
    G,i,j = bond.split('_')
    if int(i) < int(j):
        return f'G_{int(i)}_{int(j)}'
    else:
        return f'G_{int(j)}_{int(i)}'

# Decompose bonds
def decompose_bond(bond):
    
    A = bond.split('_')
    i = A[1]
    j = A[2]
    decomp = [f'G_{i}',f'G_{j}']
    return decomp

# Corner map G_i_j_k -> G_k_j_i wherer k < i
def corner_map(bond):
    G,i,j,k = bond.split('_')
    if int(i) < int(k):
        return f'G_{int(i)}_{int(j)}_{int(k)}'
    else:
        return f'G_{int(k)}_{int(j)}_{int(i)}'

# Get corner tensors around a point, this will account for all the corners in a plus shape
def get_corners(j,n):
    neighbours = next_to(j,n, T = BC)
    h_list = [p for p in neighbours if (p % n) == (j % n)]
    v_list = [p for p in neighbours if (p % n) != (j % n)]
    corners = [corner_map(f'G_{v}_{j}_{h}') for (v,h) in it.product(h_list,v_list)]
    return corners

# Get all rank 3 tensors surroundng a point.
def get_all_3_tensors(j,n):
    neighbours = next_to(j,n, T = BC)
    _3_tensors = []
    pairs = [(i,k) for i,k in it.combinations(neighbours,2)]
    for (i,k) in pairs:
        _3_tensors.append(f'G_{i}_{j}_{k}')
    return _3_tensors

# Split corners up into rank two tensors
def decompose_corners(corner):
    G,i,j,k = corner.split('_')
    decomp = [bond_map(f'G_{i}_{j}'),bond_map(f'G_{j}_{k}')]
    return decomp

# Rank 4 tensor map
def _4_tensor_map(tensor):
    parts = tensor.split('_')[1:]
    if int(parts[0]) > int(parts[-1]):
        parts.reverse()
    return(f'G_{parts[0]}_{parts[1]}_{parts[2]}_{parts[3]}')

# Get all rank 4 tensors
def get_4_tensors(j,n):
    tensors = []
    neighbours = next_to(j,n, T = BC)
    d2_neighbours =[next_to(i,n) for i in neighbours]
    for (a,b) in [(a,b) for a,b in it.combinations(neighbours,2)]:
        for k in next_to(b,n):
            tensors.append(_4_tensor_map(f'G_{a}_{j}_{b}_{k}'))
    return list(set(tensors))

# Decompose all rank 4 tensors
def decompose_4_tensors(j,n):
    G,i,j,k,l = corner.split('_')
    decomp = [corner_map(f'G_{i}_{j}_{k}'),corner_map(f'G_{j}_{k}_{l}')]
    return decomp

def plot_solution(active_coords,active_edges,n):
    n_ = n  # Size of the square lattice

    # Create a 2D grid graph
    G = nx.Graph()
    for i in active_coords:
        G.add_node(i)

    for u, v in [(i, j) for i,j in active_edges]:
        G.add_edge(u, v)

    # Draw the nodes and edges of the graph
    pos = {(i, j): (i, j) for i in range(n_) for j in range(n_)}
    nx.draw(G, pos=pos, with_labels=True, node_size=10, edge_color='gray')

    # Show the plot
    plt.axis('equal')
    plt.show()

def checking_function(N,L,sites,bonds,rank_three_tensors):
    # Check there are the correct number of sites
    if N == len(sites):
        print('Correct number of sites:', N)
    else:
        print('Worng number of sites: ', len(sites))
    # Check there are the correct number of bonds
    if L == len(bonds):
        print('Correct number of bonds:', L)
    else:
        print('Worng number of bonds: ', len(bonds))

    # Check that there are 2 sites for every bond
    for bond in bonds:
        sites_in_bond = []
        for site in sites:
            if set(bond).issuperset(set(site)):
                sites_in_bond.append(tuple(site))
                # print(site,bond,len(set(tuple(sites_in_bond))))
        if len(set(tuple(sites_in_bond))) != 2:
            print('Bond : ' , bond, 'is not consistent.','The sites in this bond are: ',sites_in_bond)

    # Check there is a bond on every site
    for site in sites:
        bonds_for_site = []
        for bond in bonds:
            if set(site).issubset(set(bond)):
                bonds_for_site.append(tuple(bond))
        if len(set(bonds_for_site)) != 1 and len(set(tuple(bonds_for_site))) != 2:
            print('Site :',site, f'is part of {len(set(tuple(bonds_for_site)))} bonds')


    # Check there are two bonds in every tensor
    for t3 in rank_three_tensors:
        for bond in bonds:
            bond_in_tensor = []
            if set(t3).issuperset(set(bond)):
                bond_in_tensor.append(tuple(bond))
            if len(set(bond_in_tensor)) > 2:
                print('Tensor :', t3, 'is not consistent with bonds : ', bond)

    for bond in bonds: 
        tensor_containing_bond = []
        for t3 in rank_three_tensors:
            if set(bond).issubset(t3):
                tensor_containing_bond.append(tuple(t3))
        if len(set(tensor_containing_bond)) != 1 and len(set(tensor_containing_bond)) != 2:
            print('Bond :', bond, f'is part of {tensor_containing_bond} bonds')

    
    # Check that there are bonds for all rank 3 tensors
    for t in rank_three_tensors:
        t = corner_map(f'G_{t[0]}_{t[1]}_{t[2]}')
        bond1, bond2 = decompose_corners(t)
        bond1 = bond1.split('_')[1:]
        bond2 = bond2.split('_')[1:]
        if (bond1 not in bonds) or (bond2 not in bonds):
            print(f'bond {bond1} or {bond2} are not contained within bonds but are in {t}' )



In [26]:
def get_random_walk_bqm(n,N,L,Am,Ab,Asa,A2,A3,A4,L4,L6,BC):
    #n is the size of the grid
    #N is the number of active sites
    #L is the number of bonds
    #Am is the monomer strength
    #Ab is the bond strength
    #A2 is the rank 1-2 tensor consistency bias
    #Asa is the self avoiding walk bias 
    #A3 is the rank 2-3 tensor consistency bias

    X = [i for i in range(n**2)]
 
    #Initialise BQM
    bqm = dimod.BinaryQuadraticModel({}, {}, 0.0, dimod.BINARY)

    #V_mon
    V_mon = combinations([f'G_{i}' for i in X], N, strength = Am)
    # print('V_mon : ', V_mon) 
    bqm.update(V_mon)
    
    #V_bond
    bond_list = get_all_bonds_global(n)
    V_bond = combinations(list(set(bond_list)), L, strength = Ab)
    # print('V_bond : ', V_bond)
    bqm.update(V_bond)

    #V_SA
    V_SA = dimod.BinaryQuadraticModel({}, {}, 0.0, dimod.BINARY)
    for j in X:
        _3_tensors = get_all_3_tensors(j,n)
        if len(_3_tensors) != 1:
            quadratic = [quad for quad in it.combinations(_3_tensors,2)]
            for a,b in it.combinations(_3_tensors,2):
                V_SA.add_quadratic(a,b,Asa)
                # print([a for a in it.combinations(_3_tensors,2)])
    # print('V_SA : ', V_SA)
    bqm.update(V_SA)


    #V_2 - consistancy relation between 2 sites and 1 bond
    V_2 = dimod.BinaryQuadraticModel({}, {}, 0.0, dimod.BINARY)
    for bond in get_all_bonds_global(n):
        i,j = decompose_bond(bond)
        V_2.add_linear(bond,A2)
        V_2.add_quadratic(bond,i, - A2/2)
        V_2.add_quadratic(bond,j, - A2/2)

    # for i in X:
    #     for j in next_to(i,n):
    #         V_2.add_linear(bond_map(f'G_{i}_{j}'),A2)
    #         V_2.add_quadratic(bond_map(f'G_{i}_{j}'),f'G_{i}', - A2)
    #         # V_2.add_quadratic(bond_map(f'G_{i}_{j}'),f'G_{j}', - A2)
    # # print('V_2 : ', V_2)
    bqm.update(V_2)


    # V_3 - consistancy relation between and walk of length 3 and the corresponding sites 
    V_3 = dimod.BinaryQuadraticModel({}, {}, 0.0, dimod.BINARY)
    for j in X:
        L = []
        for _3_tensors in get_all_3_tensors(j,n):
           binarys = decompose_corners(_3_tensors)
           V_3.add_linear(_3_tensors,A3/math.factorial(3))
           V_3.add_quadratic(binarys[0],binarys[1],A3/math.factorial(3))
           V_3.add_quadratic(_3_tensors,binarys[0],-2*A3/math.factorial(3))    
           V_3.add_quadratic(_3_tensors,binarys[1],-2*A3/math.factorial(3))    
           L.append([_3_tensors,binarys])
        #    print(j,_3_tensors,binarys)
    # print('V_3 : ', V_3)
    bqm.update(V_3)
    return bqm

# V_4 - consistancy relation between and walk of length 4 and the corresponding sites 
    V_4 = dimod.BinaryQuadraticModel({}, {}, 0.0, dimod.BINARY)
    for j in X:
        L = []
        for _4_tensors in get_4_tensors(j,n):
           binarys = decompose_4_tensors(_4_tensors)
           V_4.add_linear(_4_tensors,A3/math.factorial(3))
           V_4.add_quadratic(binarys[0],binarys[1],A4/math.factorial(3))
           V_4.add_quadratic(_4_tensors,binarys[0],-2*A4/math.factorial(3))    
           V_4.add_quadratic(_4_tensors,binarys[1],-2*A4/math.factorial(3))    
           L.append([_4_tensors,binarys])
        #    print(j,_4_tensors,binarys)
    # print('V_4 : ', V_4)
    bqm.update(V_4)
    return bqm


    # L_4 - suppressing 4-loops
    L_4 = dimod.BinaryQuadraticModel({}, {}, 0.0, dimod.BINARY)
    _3_tensors = []
    for i in X:
        _3_tensors += get_all_3_tensors(j,n)
    _3_tensors = list(set(_3_tensors))
    for tensor1, tensor2 in it.combinations(_3_tensors,2):
        s_1 = int(tensor1.split('_')[1]) #manipulate the start if the first tensor
        f_1 = int(tensor1.split('_')[-1]) #manipulate the end of the first tensor 
        s_2 = int(tensor2.split('_')[1]) #...
        f_2 = int(tensor2.split('_')[-1]) #...
        if s_1 == s_2 and f_1 == f_2 and tensor1 != tensor2:
            L_4.add_quadratic(tensor1,tensor2,L4)
    bqm.update(L_4)


    # L_6 Supress higher order loops
    L_6 = dimod.BinaryQuadraticModel({}, {}, 0.0, dimod.BINARY)
    _4_tensors = []
    for i in X:
        _4_tensors + get_4_tensors(i)
    for t1,t2 in it.combinations(_4_tensors,2):
        if t1.split('_')[1] == t2.split('_')[1] and t1.split('_')[-1] == t2.split('_')[-1]:
            L.add_quadratic(t1,t2,L6)
    bqm.update(L_6)

In [27]:
# n = 4,N = 15,L = 7,Am=1,Ab=2,Asa=30,A2=1 works very well
BC = 'Default'
# BC = 'Periodic'

n = 6
N = 20
L = 18
Am = 1
Ab= 1
Asa= 120
A2= 2
A3= 6

A4= 0
L4= 0 #24
L6= 0
                                    
random_walk_bqm = get_random_walk_bqm(n,N,L,Am,Ab,Asa,A2,A3,A4,L4,L6,BC)

# solution = KerberosSampler().sample(random_walk_bqm, num_reads = 10)

# solution_dataframe = solution.to_pandas_dataframe()

# all_solutions = []
# all_energy_num = []
# for i in range(solution_dataframe.shape[0]):
#     all_solutions.append(solution_dataframe.iloc[0,:-2].to_dict())
#     all_energy_num.append(solution_dataframe.iloc[0,-2:].to_dict())

In [28]:
def create_Hamiltonian(Ising_dict):
    """
    Function in charge of generating the Hamiltonian of the statement.

    Args:
        h (float): magnetic field strength

    Returns:
        (qml.Hamiltonian): Hamiltonian of the statement associated to h
    """
    obs = []
    coeff = []

    #Create linear terms
    for linear in Ising_dict[0].keys():
        obs.append(qml.PauliZ(linear))
        coeff.append(Ising_dict[0][linear])

    for i,j in list(Ising_dict[1].keys()):
        obs.append(qml.PauliZ(i) @ qml.PauliZ(j))
        coeff.append(Ising_dict[1][(i,j)])
    
    H = qml.Hamiltonian(coeff, obs)


    return(H)

In [29]:
create_Hamiltonian(random_walk_bqm.to_ising())

<Hamiltonian: terms=3348, wires=['G_0', 'G_0_1', 'G_0_6', 'G_0_6_7', 'G_1', 'G_10', 'G_10_11', 'G_10_16', 'G_10_16_15', 'G_10_16_17', 'G_10_4_3', 'G_10_4_5', 'G_10_9_8', 'G_11', 'G_11_10_9', 'G_11_17', 'G_11_17_16', 'G_11_5_4', 'G_12', 'G_12_13', 'G_12_18', 'G_12_18_19', 'G_12_6_0', 'G_12_6_7', 'G_13', 'G_13_14', 'G_13_19', 'G_13_19_18', 'G_13_19_20', 'G_13_7_1', 'G_13_7_6', 'G_13_7_8', 'G_14', 'G_14_13_12', 'G_14_15', 'G_14_20', 'G_14_20_19', 'G_14_20_21', 'G_14_8_2', 'G_14_8_7', 'G_14_8_9', 'G_15', 'G_15_14_13', 'G_15_16', 'G_15_21', 'G_15_21_20', 'G_15_21_22', 'G_15_9_10', 'G_15_9_3', 'G_15_9_8', 'G_16', 'G_16_10_11', 'G_16_10_4', 'G_16_10_9', 'G_16_15_14', 'G_16_17', 'G_16_22', 'G_16_22_21', 'G_16_22_23', 'G_17', 'G_17_11_10', 'G_17_11_5', 'G_17_16_15', 'G_17_23', 'G_17_23_22', 'G_18', 'G_18_12_13', 'G_18_12_6', 'G_18_19', 'G_18_24', 'G_18_24_25', 'G_19', 'G_19_13_12', 'G_19_13_14', 'G_19_13_7', 'G_19_20', 'G_19_25', 'G_19_25_24', 'G_19_25_26', 'G_1_2', 'G_1_7', 'G_1_7_6', 'G_1_7_8

In [31]:
len(create_Hamiltonian(random_walk_bqm.to_ising()).wires)

244

In [30]:
dev = qml.device("default.qubit", wires=create_Hamiltonian(random_walk_bqm.to_ising()).wires)

@qml.qnode(dev)
def model(params, H):
    """
    To implement VQE you need an ansatz for the candidate ground state!
    Define here the VQE ansatz in terms of some parameters (params) that
    create the candidate ground state. These parameters will
    be optimized later.

    Args:
        params (numpy.array): parameters to be used in the variational circuit
        H (qml.Hamiltonian): Hamiltonian used to calculate the expected value

    Returns:
        (float): Expected value with respect to the Hamiltonian H
    """

   # Put your code here #
    k = 0
    for i in H.wires: 
        qml.RZ(params[k],i)
        k+= 1
        qml.RY(params[k],i)
        k+= 1
        
    for i in H.wires: 
        if i != H.wires[0]:
            qml.CNOT([H.wires[0],i])
        
        
    for i in H.wires:
        qml.RY(params[k],i)
        k+=1
        qml.RZ(params[k],i)
        k+=1
        
    return qml.expval(H)

ValueError: Maximum allowed dimension exceeded

In [12]:
init_params = np.random.random(4*len(create_Hamiltonian(random_walk_bqm.to_ising()).wires))
model(init_params,create_Hamiltonian(random_walk_bqm.to_ising()))

tensor(-3.74112454, requires_grad=True)

In [14]:
def train(Ising_dict):
    """
    In this function you must design a subroutine that returns the
    parameters that best approximate the ground state.

    Args:
        h (float): magnetic field strength

    Returns:
        (numpy.array): parameters that best approximate the ground state.
    """
    
    H = create_Hamiltonian(Ising_dict)
    
    
    init_params = np.random.random(4*len(Ising_dict[0]))
 
    def cost_fn(params):
        return model(params, H)
        
    
    opt = qml.GradientDescentOptimizer(stepsize=0.01)

    params = init_params

    gd_param_history = [params]
    gd_cost_history = []
    max_iterations = 10000
    conv_tol = 1e-06


    for n in range(max_iterations):

        # Take step
        params, prev_energy = opt.step_and_cost(cost_fn, params)
        gd_param_history.append(params)
        gd_cost_history.append(prev_energy)

        energy = cost_fn(params)

        # Calculate difference between new and old energies
        conv = np.abs(energy - prev_energy)

        # if n % 20 == 0:
        #     print(
        #         "Iteration = {:},  Energy = {:.8f} Ha,  Convergence parameter = {"
        #         ":.8f} Ha".format(n, energy, conv)
        #     )

        if conv <= conv_tol:
            break

#         print()
#         print("Final value of the energy = {:.8f} Ha".format(energy))
#         print("Number of iterations = ", n)
        
    return params



In [15]:
params = train(random_walk_bqm.to_ising())

tensor([ 8.28963335e-01,  3.98626896e-11,  4.00815389e-01,
        -2.11307118e-01,  7.97102886e-01,  1.47083143e+00,
         2.38933029e-01,  2.51664603e-01,  9.53431987e-01,
        -9.29439903e-02,  8.18579261e-01,  3.91165975e-01,
         5.64203151e-01, -1.90999628e-01,  2.78642779e-01,
         3.43030508e-01,  9.84202554e-01, -1.24925969e-01,
         4.37461472e-02, -1.79724805e-02,  6.93092118e-01,
         1.80400079e-01,  1.87842506e-01,  9.55894749e-02,
         6.90660678e-02,  7.99056105e-01,  2.11310676e-01,
         3.08451103e-01,  1.66656926e+00,  6.89389299e-01,
        -2.51527047e-01,  8.81805410e-01,  9.30367210e-02,
         4.29183909e-01, -3.86507285e-01,  7.12751267e-01,
         1.90999632e-01,  6.96909085e-02,  3.83113253e-01,
         4.95217051e-01,  1.24968366e-01,  5.18655198e-01,
         1.80080042e-02,  2.45250709e-01, -1.80335376e-01,
         5.77396178e-02, -9.55894743e-02,  7.34656617e-01], requires_grad=True)

In [22]:
dev = qml.device("default.qubit", wires=create_Hamiltonian(random_walk_bqm.to_ising()).wires, shots = 100)

@qml.qnode(dev)
def circuit(params, H):
    """
    To implement VQE you need an ansatz for the candidate ground state!
    Define here the VQE ansatz in terms of some parameters (params) that
    create the candidate ground state. These parameters will
    be optimized later.

    Args:
        params (numpy.array): parameters to be used in the variational circuit
        H (qml.Hamiltonian): Hamiltonian used to calculate the expected value

    Returns:
        (float): Expected value with respect to the Hamiltonian H
    """

   # Put your code here #
    k = 0
    for i in H.wires: 
        qml.RZ(params[k],i)
        k+= 1
        qml.RY(params[k],i)
        k+= 1
        
    for i in H.wires: 
        if i != H.wires[0]:
            qml.CNOT([H.wires[0],i])
        
        
    for i in H.wires:
        qml.RY(params[k],i)
        k+=1
        qml.RZ(params[k],i)
        k+=1

    return [qml.sample(qml.PauliZ(i)) for i in H.wires]

In [23]:
ground_state = circuit(params, create_Hamiltonian(random_walk_bqm.to_ising()))

tensor([[ 1,  1,  1, ...,  1,  1,  1],
        [ 1,  1,  1, ...,  1,  1,  1],
        [-1, -1, -1, ..., -1, -1, -1],
        ...,
        [ 1,  1,  1, ...,  1,  1,  1],
        [ 1,  1,  1, ...,  1,  1,  1],
        [ 1,  1,  1, ...,  1,  1,  1]], requires_grad=True)